# Introduction

For my Machine Learing class at A&M, I implemented common ML algorithms from the ground up. It's amazing how easy it is to implement an ML technique in python. I thought that I'd share my implementations in a series of short blog posts. (I am new to python, so please cut me some slack.) In my opinion, there are three main advantages to this endeavour:

- The understanding we get when we implement something from scratch is unmatched. A lot of the times, we think we understand a concept but when we implement it, we get a slew of doubts, thereby forcing us to really get the details right. 
- It helps us appreciate the design of libraries better and understand why things are designed a certain way. It helps truly get the API of a library with enduring success like scikit-learn.  
- The satisfaction when we see the identical results convinces, that these libraries do not do something like the algorithm we study but actually implement the very algorithm we study, only better. 


# Techniques

In this blog, I am sharing the code for 

1. A small module to scale training and testing dataset features, the right way.
2. A K Nearest Neighbour classifier

# Setup (Libraries and Data)

In [9]:
import pandas as pd
import numpy as np
import scipy

Since this series of posts concentrates on the implementations, I won't delve into the data and results per se. I'll just briefly explain the dataset and the classification task for demonstration purposes. The dataset shows the response - area (in square feet) affected by volcanos and different features corresponding to it. We can change this to a classification problem by predicting whether the area was affected by volcanoes or not (area > 0). 

In [8]:
df_train_org = pd.read_csv("../data/ml_scratch_1_train.csv")
df_test_org = pd.read_csv("../data/ml_scratch_1_test.csv")
print(df_train_org.head())

tr_X = df_train_org.loc[:, df_train_org.columns != 'area']
ts_X = df_test_org.loc[:, tr_X.columns]
tr_y = (df_train_org['area'] > 0)*1
ts_y = (df_test_org['area'] > 0)*1

   X  Y  month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5      3    5  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4     10    2  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4     10    6  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6      3    5  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6      3    7  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0


#

In [ ]:
class Knn():

    """A knn classifier """

    def __init__(self, 
                 k:"int" = 3,
                 norm:"float"=2.0 ,
                 standardization="normalize"):

        self.k = k
        self.norm = norm
        self.standardization = standardization
  

    def fit(self, tr_X, tr_y):
        """Pass training data as X and y """
        self.standardizer = Standardize(X=tr_X)
        self.tr_X = self.standardizer.standardize(method=self.standardization)
        self.tr_y = tr_y


    def predict(self, ts_X):
        ts_X = self.standardizer.standardize(
            method=self.standardization, X=ts_X)
        d = self._dist_matrix(ts_X)
        neighbours = self._get_nearest_neighbour_indices(d)
        counts = self._count_votes(neighbours)
        return self._tag_winner(counts)


    def _dist_matrix(self, ts_X):
        dists = scipy.spatial.distance_matrix(ts_X, self.tr_X, p=self.norm)
        dists = np.vstack(dists)
        return dists


    def _calc_dist(self, vec):
        rows = self.tr_X.shape[0]
        dists = np.zeros(rows)
        for i in range(rows):
            tr_row = self.tr_X.iloc[i]
            dist = 0
            for j in range(len(tr_row)):
                if isinstance(tr_row[j].dtype, int) and self.cat_hamming:
                    dist += (tr_row[j] == vec[j])*1
                else:
                    dist += (abs(tr_row[j] - vec[j])**self.norm)
            dists[i] = dist ** (1/self.norm)
        return dists


    def _get_nearest_neighbour_indices(self, dist_matrix):
        return np.argpartition(dist_matrix, kth=self.k, axis=1)[:, :self.k]


    def _count_votes(self, indice_matrix):
        return np.apply_along_axis(lambda x: collections.Counter(self.tr_y.iloc[x]), 1, indice_matrix)


    def _tag_winner(self, vote_mat):
        count_max = np.vectorize(lambda x: x.most_common(1)[0][0])
        return count_max(vote_mat)


    def score(self, ts_X, ts_y):
        return (ts_y == self.predict(ts_X)).mean()


    def __repr__(self):
        return (f'Knn(k={self.k}, norm={self.norm}, standardization="{self.standardization}"')
